In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [25]:
clean_df = pd.read_csv('merged_dataset1.csv')

In [26]:
data=clean_df

# Convert 'Date_Rptd' and 'DATE_OCC' to datetime
# data['Date_Rptd'] = pd.to_datetime(data['Date_Rptd'], format='%m/%d/%Y %I:%M:%S %p')
# data['DATE_OCC'] = pd.to_datetime(data['DATE_OCC'], format='%Y-%m-%d')


# # Extract day of week, month, and year from 'DATE_OCC'
# data['Day_of_Week'] = data['DATE_OCC'].dt.dayofweek
# data['Month'] = data['DATE_OCC'].dt.month
# data['Year'] = data['DATE_OCC'].dt.year

# # Drop the original 'Date_Rptd' and 'DATE_OCC' columns
# data = data.drop(['Date_Rptd', 'DATE_OCC'], axis=1)

# # Convert 'Vict_Sex' and 'Vict_Descent' to dummy variables
# categorical_to_convert = ['Vict_Sex', 'Region_Ethnic_Origin', 'AREA']
# data = pd.get_dummies(data, columns=categorical_to_convert, drop_first=True)
# data['Day_of_Week'] = data['Day_of_Week'].astype('category')
# data['Month'] = data['Month'].astype('category')
# data['Year'] = data['Year'].astype('category')

# # Create dummy variables for these columns
# data = pd.get_dummies(data, columns=['Day_of_Week', 'Month', 'Year'], drop_first=True)


# # We will not convert 'LOCATION' due to its high cardinality
# data.drop(['LOCATION', 'Vict_Descent', 'DR_NO'], axis=1, inplace=True)

In [27]:
clean_df

,DR_NO,Date_Rptd,DATE_OCC,TIME_OCC,AREA,AREA_NAME,Rpt_Dist_No,Part_1-2,Crm_Cd,Crm_Cd_Desc,...,Max_Humidity,Avg_Humidity,Min_Humidity,Max_Windspeed,Avg_Windspeed,Min_Windspeed,Max_Pressure,Avg_Pressure,Min_Pressure,Total_Precipitation
0,1307355,02/20/2010 12:00:00 AM,2010-02-20,1350,13,Newton,1385,2,900,VIOLATION OF COURT ORDER,...,83.000,73.200,62.000,17.000,9.700,0.000,29.800,29.800,29.700,0.000
1,100100638,02/20/2010 12:00:00 AM,2010-02-20,100,1,Central,164,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",...,83.000,73.200,62.000,17.000,9.700,0.000,29.800,29.800,29.700,0.000
2,100100639,02/20/2010 12:00:00 AM,2010-02-20,230,1,Central,164,2,626,INTIMATE PARTNER - SIMPLE ASSAULT,...,83.000,73.200,62.000,17.000,9.700,0.000,29.800,29.800,29.700,0.000
3,100106650,02/20/2010 12:00:00 AM,2010-02-20,45,1,Central,155,2,624,BATTERY - SIMPLE ASSAULT,...,83.000,73.200,62.000,17.000,9.700,0.000,29.800,29.800,29.700,0.000
4,100106654,02/20/2010 12:00:00 AM,2010-02-20,1130,1,Central,146,2,946,OTHER MISCELLANEOUS CRIME,...,83.000,73.200,62.000,17.000,9.700,0.000,29.800,29.800,29.700,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2886525,220414089,10/08/2022 12:00:00 AM,2022-10-06,900,4,Hollenbeck,406,1,310,BURGLARY,...,84.000,79.500,71.000,15.000,6.400,0.000,29.900,29.900,29.800,0.000
2886526,220122081,10/07/2022 12:00:00 AM,2022-10-06,2300,1,Central,176,1,330,BURGLARY FROM VEHICLE,...,84.000,79.500,71.000,15.000,6.400,0.000,29.900,29.900,29.800,0.000
2886527,220414017,10/06/2022 12:00:00 AM,2022-10-06,1800,4,Hollenbeck,443,1,330,BURGLARY FROM VEHICLE,...,84.000,79.500,71.000,15.000,6.400,0.000,29.900,29.900,29.800,0.000
2886528,220318042,10/07/2022 12:00:00 AM,2022-10-06,1330,3,Southwest,374,1,440,THEFT PLAIN - PETTY ($950 & UNDER),...,84.000,79.500,71.000,15.000,6.400,0.000,29.900,29.900,29.800,0.000


In [28]:
def calculate_outliers(column):
    Q1 = column.quantile(0.25)
    Q3 = column.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return ((column < lower_bound) | (column > upper_bound)).sum()

In [29]:
statistics = data.describe()

In [30]:
full_statistics = data.describe(include='all')

In [31]:
statistics

,DR_NO,TIME_OCC,AREA,Rpt_Dist_No,Part_1-2,Crm_Cd,Vict_Age,Premis_Cd,Weapon_Used_Cd,Crm_Cd_1,...,Max_Humidity,Avg_Humidity,Min_Humidity,Max_Windspeed,Avg_Windspeed,Min_Windspeed,Max_Pressure,Avg_Pressure,Min_Pressure,Total_Precipitation
count,2886530.000,2886530.000,2886530.000,2886530.000,2886530.000,2886530.000,2886530.000,2886457.000,977023.000,2886510.000,...,2886451.000,2886451.000,2886451.000,2886451.000,2886451.000,2886451.000,2886451.000,2886451.000,2886451.000,2886386.000
mean,162037612.397,1355.471,10.818,1128.261,1.438,505.882,31.180,311.477,370.285,505.695,...,85.088,69.383,49.896,15.446,7.158,0.451,29.898,29.830,29.349,0.024
std,41994106.943,648.315,6.039,603.911,0.496,210.209,20.937,212.941,115.667,210.061,...,12.055,15.909,20.225,13.291,2.627,1.353,0.110,0.192,3.632,0.161
min,817.000,1.000,1.000,100.000,1.000,110.000,-12.000,101.000,101.000,110.000,...,19.000,12.500,0.000,0.000,0.000,0.000,29.300,26.500,0.000,0.000
25%,121819083.500,922.000,6.000,629.000,1.000,330.000,19.000,102.000,400.000,330.000,...,81.000,64.500,37.000,13.000,5.700,0.000,29.800,29.800,29.700,0.000
50%,160911884.500,1430.000,11.000,1142.000,1.000,442.000,31.000,210.000,400.000,442.000,...,87.000,74.000,57.000,15.000,6.900,0.000,29.900,29.800,29.800,0.000
75%,191214589.750,1900.000,16.000,1638.000,2.000,626.000,46.000,501.000,400.000,626.000,...,93.000,79.700,65.000,17.000,8.100,0.000,30.000,29.900,29.900,0.000
max,910220366.000,2359.000,21.000,2199.000,2.000,956.000,118.000,976.000,516.000,999.000,...,100.000,98.200,96.000,807.000,94.700,17.000,30.400,30.300,30.200,3.000


In [32]:
full_statistics

,DR_NO,Date_Rptd,DATE_OCC,TIME_OCC,AREA,AREA_NAME,Rpt_Dist_No,Part_1-2,Crm_Cd,Crm_Cd_Desc,...,Max_Humidity,Avg_Humidity,Min_Humidity,Max_Windspeed,Avg_Windspeed,Min_Windspeed,Max_Pressure,Avg_Pressure,Min_Pressure,Total_Precipitation
count,2886530.000,2886530,2886530,2886530.000,2886530.000,2886530,2886530.000,2886530.000,2886530.000,2886530,...,2886451.000,2886451.000,2886451.000,2886451.000,2886451.000,2886451.000,2886451.000,2886451.000,2886451.000,2886386.000
unique,NaN,5129,5113,NaN,NaN,21,NaN,NaN,NaN,144,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,02/02/2023 12:00:00 AM,2011-01-01,NaN,NaN,77th Street,NaN,NaN,NaN,BATTERY - SIMPLE ASSAULT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,1571,4037,NaN,NaN,186509,NaN,NaN,NaN,255818,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,162037612.397,NaN,NaN,1355.471,10.818,NaN,1128.261,1.438,505.882,NaN,...,85.088,69.383,49.896,15.446,7.158,0.451,29.898,29.830,29.349,0.024
std,41994106.943,NaN,NaN,648.315,6.039,NaN,603.911,0.496,210.209,NaN,...,12.055,15.909,20.225,13.291,2.627,1.353,0.110,0.192,3.632,0.161
min,817.000,NaN,NaN,1.000,1.000,NaN,100.000,1.000,110.000,NaN,...,19.000,12.500,0.000,0.000,0.000,0.000,29.300,26.500,0.000,0.000
25%,121819083.500,NaN,NaN,922.000,6.000,NaN,629.000,1.000,330.000,NaN,...,81.000,64.500,37.000,13.000,5.700,0.000,29.800,29.800,29.700,0.000
50%,160911884.500,NaN,NaN,1430.000,11.000,NaN,1142.000,1.000,442.000,NaN,...,87.000,74.000,57.000,15.000,6.900,0.000,29.900,29.800,29.800,0.000
75%,191214589.750,NaN,NaN,1900.000,16.000,NaN,1638.000,2.000,626.000,NaN,...,93.000,79.700,65.000,17.000,8.100,0.000,30.000,29.900,29.900,0.000


In [33]:
transposed_statistics = statistics.T

In [34]:
transposed_statistics

,count,mean,std,min,25%,50%,75%,max
DR_NO,2886530.000,162037612.397,41994106.943,817.000,121819083.500,160911884.500,191214589.750,910220366.000
TIME_OCC,2886530.000,1355.471,648.315,1.000,922.000,1430.000,1900.000,2359.000
AREA,2886530.000,10.818,6.039,1.000,6.000,11.000,16.000,21.000
Rpt_Dist_No,2886530.000,1128.261,603.911,100.000,629.000,1142.000,1638.000,2199.000
Part_1-2,2886530.000,1.438,0.496,1.000,1.000,1.000,2.000,2.000
Crm_Cd,2886530.000,505.882,210.209,110.000,330.000,442.000,626.000,956.000
Vict_Age,2886530.000,31.180,20.937,-12.000,19.000,31.000,46.000,118.000
Premis_Cd,2886457.000,311.477,212.941,101.000,102.000,210.000,501.000,976.000
Weapon_Used_Cd,977023.000,370.285,115.667,101.000,400.000,400.000,400.000,516.000
Crm_Cd_1,2886510.000,505.695,210.061,110.000,330.000,442.000,626.000,999.000


In [35]:
transposed_full_statistics = full_statistics.T

In [36]:
transposed_full_statistics

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
DR_NO,2886530.000,NaN,NaN,NaN,162037612.397,41994106.943,817.000,121819083.500,160911884.500,191214589.750,910220366.000
Date_Rptd,2886530,5129,02/02/2023 12:00:00 AM,1571,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DATE_OCC,2886530,5113,2011-01-01,4037,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TIME_OCC,2886530.000,NaN,NaN,NaN,1355.471,648.315,1.000,922.000,1430.000,1900.000,2359.000
AREA,2886530.000,NaN,NaN,NaN,10.818,6.039,1.000,6.000,11.000,16.000,21.000
AREA_NAME,2886530,21,77th Street,186509,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rpt_Dist_No,2886530.000,NaN,NaN,NaN,1128.261,603.911,100.000,629.000,1142.000,1638.000,2199.000
Part_1-2,2886530.000,NaN,NaN,NaN,1.438,0.496,1.000,1.000,1.000,2.000,2.000
Crm_Cd,2886530.000,NaN,NaN,NaN,505.882,210.209,110.000,330.000,442.000,626.000,956.000
Crm_Cd_Desc,2886530,144,BATTERY - SIMPLE ASSAULT,255818,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
continuous_vars = data.select_dtypes(include=['int64', 'float64']).columns
categorical_vars = data.select_dtypes(include=['object', 'category']).columns
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [38]:
continuous_stats = data[continuous_vars].describe().transpose()
print("Continuous Variables Descriptive Statistics:")
print(continuous_stats)

Continuous Variables Descriptive Statistics:
                          count          mean          std      min   
DR_NO               2886530.000 162037612.397 41994106.943  817.000  \
TIME_OCC            2886530.000      1355.471      648.315    1.000   
AREA                2886530.000        10.818        6.039    1.000   
Rpt_Dist_No         2886530.000      1128.261      603.911  100.000   
Part_1-2            2886530.000         1.438        0.496    1.000   
Crm_Cd              2886530.000       505.882      210.209  110.000   
Vict_Age            2886530.000        31.180       20.937  -12.000   
Premis_Cd           2886457.000       311.477      212.941  101.000   
Weapon_Used_Cd       977023.000       370.285      115.667  101.000   
Crm_Cd_1            2886510.000       505.695      210.061  110.000   
Crm_Cd_2             193224.000       952.455      120.555  122.000   
Crm_Cd_3               5062.000       975.353       78.331   93.000   
Crm_Cd_4                151.000 

In [39]:
categorical_stats = pd.DataFrame(index=categorical_vars, columns=['Count', 'Unique', 'Top', 'Frequency'])

for var in categorical_vars:
    categorical_stats.loc[var, 'Count'] = data[var].count()
    categorical_stats.loc[var, 'Unique'] = data[var].nunique()
    categorical_stats.loc[var, 'Top'] = data[var].mode()[0]
    categorical_stats.loc[var, 'Frequency'] = data[var].value_counts().iloc[0]

print("Categorical Variables Descriptive Statistics:")
print(categorical_stats)

Categorical Variables Descriptive Statistics:
                Count  Unique                                             Top   
Date_Rptd     2886530    5129                          02/02/2023 12:00:00 AM  \
DATE_OCC      2886530    5113                                      2011-01-01   
AREA_NAME     2886530      21                                     77th Street   
Crm_Cd_Desc   2886530     144                        BATTERY - SIMPLE ASSAULT   
Mocodes       2556356  626351                                            0344   
Vict_Sex      2595169       6                                               M   
Vict_Descent  2595112      20                                               H   
Premis_Desc   2885861     320                                          STREET   
Weapon_Desc    977022      79  STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)   
Status        2886526       9                                              IC   
Status_Desc   2886530       6                                  

In [40]:
continuous_vars = data.select_dtypes(include=['int64', 'float64']).columns
outliers_count = {col: calculate_outliers(data[col]) for col in continuous_vars}

# Adding the number of outliers to the descriptive statistics table
for col in continuous_vars:
    transposed_statistics.loc[col, 'Outliers'] = outliers_count[col]

In [41]:
transposed_statistics['Skewness'] = data[continuous_vars].skew()
transposed_statistics['Kurtosis'] = data[continuous_vars].kurtosis()
transposed_statistics['Missing Values'] = data[continuous_vars].isnull().sum()

In [42]:
# transposed_statistics['Skewness'] = data.skew()
# transposed_statistics['Kurtosis'] = data.kurtosis()
# transposed_statistics['Missing Values'] = data.isnull().sum()

# # Add Outliers (using the previously defined calculate_outliers function)
# transposed_statistics['Outliers'] = transposed_statistics.index.map(lambda x: calculate_outliers(data[x]) if x in continuous_vars else None)

# # Setting display format to avoid scientific notation
# pd.set_option('display.float_format', lambda x: '%.2f' % x)

# print(transposed_statistics)

In [43]:
numeric_cols = data.select_dtypes(include=[np.number]).columns

# Calculate descriptive statistics for numeric columns only
numeric_stats = data[numeric_cols].describe().transpose()

# Add skewness and kurtosis
numeric_stats['Skewness'] = data[numeric_cols].skew()
numeric_stats['Kurtosis'] = data[numeric_cols].kurtosis()

# Function to calculate outliers
def calculate_outliers(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return ((series < lower_bound) | (series > upper_bound)).sum()

# Add number of outliers
numeric_stats['Outliers'] = data[numeric_cols].apply(calculate_outliers)

# Display the adjusted statistics table
print("Adjusted Numeric Variables Descriptive Statistics:")
print(numeric_stats)

Adjusted Numeric Variables Descriptive Statistics:
                          count          mean          std      min   
DR_NO               2886530.000 162037612.397 41994106.943  817.000  \
TIME_OCC            2886530.000      1355.471      648.315    1.000   
AREA                2886530.000        10.818        6.039    1.000   
Rpt_Dist_No         2886530.000      1128.261      603.911  100.000   
Part_1-2            2886530.000         1.438        0.496    1.000   
Crm_Cd              2886530.000       505.882      210.209  110.000   
Vict_Age            2886530.000        31.180       20.937  -12.000   
Premis_Cd           2886457.000       311.477      212.941  101.000   
Weapon_Used_Cd       977023.000       370.285      115.667  101.000   
Crm_Cd_1            2886510.000       505.695      210.061  110.000   
Crm_Cd_2             193224.000       952.455      120.555  122.000   
Crm_Cd_3               5062.000       975.353       78.331   93.000   
Crm_Cd_4                15

In [44]:
data['TIME_OCC'].describe()

count   2886530.000
mean       1355.471
std         648.315
min           1.000
25%         922.000
50%        1430.000
75%        1900.000
max        2359.000
Name: TIME_OCC, dtype: float64

In [ ]:
# Export numeric variables descriptive statistics to CSV
numeric_stats.to_csv('numeric_stats.csv', index=True)

# Export categorical variables descriptive statistics to CSV
categorical_stats.to_csv('categorical_stats.csv', index=True)